In [ ]:
import random
import sys
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.spatial.distance import cdist, pdist, euclidean
import simulation.bsim as bsim
from matplotlib import animation, rc, rcParams
rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML
from matplotlib.artist import Artist

In [ ]:
import importlib
#reload this if you have made any changed to bsim
importlib.reload(bsim)

In [ ]:
##### parameters and creating environment #####

# dimensions of the map (height and width)
dim_h = 50
dim_w = 130

swarmsize = 20 
swarm = bsim.swarm()
swarm.size = swarmsize

# if verbose is True, information on collected and dropped off boxes
swarm.verbose = False

# max load = max number of boxes a robutt can carry
swarm.max_load = 20
# load_threshold = once max_load*threshold reached, start returning to base
swarm.load_threshold = 0.75
swarm.gen_agents()

# Set simulation duration
timesteps = 1000

# position of the base
base = bsim.base()
base.pos = [-100,0]
# radius of dropoff area
base.radius = 10
# radius of allowed movement
base.move_radius = 50
# speed of base when moving (see animation loop for movement times)
base_speed = 0.2

# Create the environment
env = bsim.map()
env.empty(dim_w, dim_h)
env.gen()
swarm.map = env

# Create the set of boxes to collect
boxes = bsim.boxes()
box_number = 2000
boxes.set_state('random', dim_w, dim_h, box_number)

# collection radius
boxes.radius = 1.5
# probability of collection 
boxes.prob_pickup = 1
# probability of drop-off within radius
self.prob_dropoff = 0.2




noise = 0.1
##### parameters and creating environment #####
   

def move(swarm, noise):
    mode = np.transpose(np.vstack((swarm.holding, swarm.holding)))
    mode[mode < swarm.max_load*swarm.load_threshold] = 0
    mode[mode > 0] = 1
    mode[swarm.lost == 1] = 0
    # 1 if needing to return to base, 0 otherwise
    bsim.return_to_base(swarm, base, noise, mode)
    
    # lost robutts don't move
    mode[swarm.lost == 1] = 1
    
    # 1 if doing random walk, 0 otherwise
    bsim.random_walk(swarm, boxes, base, noise, 1- mode)  
    


fontsize = 12

# First set up the figure, the axis, and the plot element we want to animate
fig, ax1 = plt.subplots(nrows=1,ncols=1, figsize=(21,8), dpi=70, facecolor='w', edgecolor='k')
plt.close()

ax1.set_xlim((-dim_w, dim_w))
ax1.set_ylim((-dim_h-10, dim_h))

# draw the beach
ax1.set_facecolor('xkcd:beige')
beach = np.arange(-dim_w, dim_w+5, 1);
wave   = 0.5*np.sin(beach)
bottom = np.zeros((len(wave)))
ax1.fill_between(beach, -dim_h-10, wave-dim_h)

# Set how data is plotted within animation loop
global line, line1
# plots red agents
line_b, = ax1.plot([], [], 'rh', markersize = 10, markeredgecolor="black", alpha = 0.9 )
# plots green agents
line, = ax1.plot([], [], 'gh', markersize = 10, markeredgecolor="black", alpha = 0.9 )
# plots white agents
line_w, = ax1.plot([], [], 'wh', markersize = 10, markeredgecolor="black", alpha = 0.9 )
# plots lost agents
line_l, = ax1.plot([], [], 'kh', markersize = 10, markeredgecolor="black", alpha = 0.9 )
# plots blue boxes
line1, = ax1.plot([], [], 's', color = 'xkcd:orange', markersize = 3, markeredgecolor="xkcd:orange", alpha = 0.95)
# plots black boxes (collected)
line1_b, = ax1.plot([], [], 'ks', markersize = 3, markeredgecolor="black", alpha = 0.8)

d_base = plt.Circle(base.pos, 2, color = 'k')
rad = plt.Circle(base.pos, base.radius, fill=False)
mrad = plt.Circle(base.pos, base.move_radius, fill=False, alpha = 0.5)
ax1.add_patch(rad)
ax1.add_patch(d_base)
ax1.add_patch(mrad)



line.set_data([], [])
line1.set_data([], [])

def init():
    global start 
    start = False
    line.set_data([], [])
    line_b.set_data([],[])
    line_w.set_data([],[])
    line_l.set_data([],[])
    line1.set_data([], [])
    line1_b.set_data([], [])
    return ([line, line_b, line_w, line_l, line1, line1_b])



field, grid = bsim.potentialField_map(swarm.map) 
swarm.field = field
swarm.grid = grid

box_data = []
done_data = []
time_data = []
heat_data = []



def animate(i):
    global start, final_i
    
    
    move(swarm, noise)
    
    swarm.get_state()
    
    score, done, demand = boxes.get_state()
    boxes.update_boxes(swarm, base, i)
    
    ## change this section to control movement of the base
    ## base.dropoffs counts number of times a robutt empties their bin
    if base.dropoffs == 25 or base.dropoffs == 100:
        final_i = i + 500
        start = True
    
    if start == True:
        boxes.update_pos(base_speed)
        base.update_pos(base_speed)
        
        if i == final_i:
            start = False

    
    patches = []
    
    d_base.center = base.pos
    rad.center = base.pos
    mrad.center = base.pos
    
    patches.append(d_base)
    patches.append(rad)
    patches.append(mrad)
    
    box_data.append(100*(score/len(boxes.boxes)))
    done_data.append(100*(done/len(boxes.boxes)))
    
   
    time_data.append(i)
    
    bdist = cdist(boxes.boxes, [base.pos])
    mask = np.logical_and(boxes.collected.flatten() == 1, bdist.flatten() < base.radius)

    line1.set_data(boxes.boxes.T[0][boxes.collected == 0], boxes.boxes.T[1][boxes.collected == 0])
    
    line1_b.set_data(boxes.boxes.T[0][mask], boxes.boxes.T[1][mask])
    line.set_data(swarm.agents.T[0][swarm.holding <= swarm.load_threshold*swarm.max_load], swarm.agents.T[1][swarm.holding <= swarm.load_threshold*swarm.max_load])
    line_b.set_data(swarm.agents.T[0][swarm.holding > swarm.load_threshold*swarm.max_load], swarm.agents.T[1][swarm.holding > swarm.load_threshold*swarm.max_load])
    line_w.set_data(swarm.agents.T[0][swarm.holding >= swarm.max_load], swarm.agents.T[1][swarm.holding >= swarm.max_load])
    line_l.set_data(swarm.agents.T[0][swarm.lost == 1], swarm.agents.T[1][swarm.lost == 1])

    
    if i == timesteps - 1:
        print('boxes collected:', score)
        print('percentage of boxes collected:', 100*score/len(boxes.boxes))
    
    return ([line, line_b, line_w, line1, line1_b]+ patches)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                            frames=timesteps, interval=100, blit=True, cache_frame_data = False)

HTML(anim.to_jshtml())

### comment the above line and uncomment the below to save as gif
#print('saving!')
#f = r"C://Users/folder/location/animation.gif" 
#f = r"C://Users/elena/Documents/Bristol/year4/bioAI/animation_full.gif"
#writergif = animation.PillowWriter(fps=60) 
#anim.save(f, writer=writergif)